In [3]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from web3 import Web3
import json
import time

# Load environment variables
load_dotenv()

# Connect to the Base network
w3 = Web3(Web3.HTTPProvider(os.getenv('RPC_URL')))

# Load contract ABIs
with open('AuctionHouse_ABI.json', 'r') as abi_file:
    auction_house_abi = json.load(abi_file)
    
with open('AuctionHouseFactory_ABI.json', 'r') as abi_file:
    auction_house_factory_abi = json.load(abi_file)
    
with open('AffiliateEscrow_ABI.json', 'r') as abi_file:
    affiliate_escrow_abi = json.load(abi_file)
    
    
with open('AuctionItemERC721_ABI.json', 'r') as abi_file:
    auction_item_erc721_abi = json.load(abi_file)
    
with open('AuctionItemERC721Factory_ABI.json', 'r') as abi_file:
    auction_item_erc721_factory_abi = json.load(abi_file)
    
with open('ERC20_ABI.json', 'r') as abi_file:
    erc20_abi = json.load(abi_file)

# Set up contract addresses
AUCTION_HOUSE_FACTORY_ADDRESS = os.getenv('AUCTION_HOUSE_FACTORY_ADDRESS')
AUCTION_ITEM_ERC721_FACTORY_ADDRESS = os.getenv('AUCTION_ITEM_ERC721_FACTORY_ADDRESS')

# Set up contract instances
auction_house_factory = w3.eth.contract(
    address=AUCTION_HOUSE_FACTORY_ADDRESS, 
    abi=auction_house_factory_abi
)


auction_item_erc721_factory = w3.eth.contract(
    address=AUCTION_ITEM_ERC721_FACTORY_ADDRESS, 
    abi=auction_item_erc721_factory_abi
)

# Set up accounts
owner_account = w3.eth.account.from_key(os.getenv('OWNER_PRIVATE_KEY'))
seller_account = w3.eth.account.from_key(os.getenv('SELLER_PRIVATE_KEY'))
bidder1_account = w3.eth.account.from_key(os.getenv('BIDDER1_PRIVATE_KEY'))
bidder2_account = w3.eth.account.from_key(os.getenv('BIDDER2_PRIVATE_KEY'))
affiliate_account = w3.eth.account.from_key(os.getenv('AFFILIATE_PRIVATE_KEY'))
arbiter_account = w3.eth.account.from_key(os.getenv('ARBITER_PRIVATE_KEY'))

# Example encrypted message structure for bids
EXAMPLE_MESSAGE = {
    "encryptedData": "0x",
    "ephemeralPublicKey": "0x",
    "iv": "0x",
    "verificationHash": "0x"
}

# Constants for the auction system
DEFAULT_SETTLEMENT_DEADLINE = 21 * 24 * 60 * 60  # 21 days in seconds

In [25]:
def create_auction_house(name, image, description, sender_account, custom_deadline=0):
    """Create a new auction house using the factory"""
    nonce = w3.eth.get_transaction_count(sender_account.address)
    
    txn = auction_house_factory.functions.createAuctionHouse(
        name,
        image,
        description,
        custom_deadline
    ).build_transaction({
        'from': sender_account.address,
        'nonce': nonce,
        'gas': 5000000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = sender_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Transaction receipt: {tx_receipt}")
    
    # Process logs to get the new auction house address
    for log in tx_receipt['logs']:
        try:
            event = auction_house_factory.events.AuctionHouseCreated().process_log(log)
            if event:
                return event['args']['auctionHouse']
        except Exception as e:
            print(f"Error processing log: {e}")
            
    print("No AuctionHouseCreated event found in the transaction logs")
    return None

def create_auction_item_erc721(name, symbol, contract_uri, sender_account):
    """Create a new AuctionItemERC721 contract using the factory"""
    nonce = w3.eth.get_transaction_count(sender_account.address)
    
    txn = auction_item_erc721_factory.functions.createAuctionItemERC721(
        name,
        symbol,
        contract_uri
    ).build_transaction({
        'from': sender_account.address,
        'nonce': nonce,
        'gas': 5000000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = sender_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Print all logs for debugging
    print("Transaction receipt logs:")
    for i, log in enumerate(tx_receipt['logs']):
        print(f"Log {i}: {log}")
    
    # Try different approaches to extract the token contract address
    
    # Approach 1: Try to find the event by name in different formats
    event_names = [
        "AuctionItemERC721Created", 
        "AuctionItemCreated", 
        "TokenContractCreated",
        "Created"
    ]
    
    for event_name in event_names:
        try:
            if hasattr(auction_item_erc721_factory.events, event_name):
                event_processor = getattr(auction_item_erc721_factory.events, event_name)()
                for log in tx_receipt['logs']:
                    try:
                        event = event_processor.process_log(log)
                        if event:
                            # Try different argument names
                            for arg_name in ['tokenContract', 'token', 'contract', 'contractAddress', 'address']:
                                if arg_name in event['args']:
                                    token_address = event['args'][arg_name]
                                    print(f"Found token address in event {event_name}, arg {arg_name}: {token_address}")
                                    return token_address
                    except Exception as e:
                        continue
        except Exception as e:
            continue
    
    # Approach 2: If we can't find the event, try to extract from the last log
    # Often the created contract address is in the last log
    if tx_receipt['logs']:
        last_log = tx_receipt['logs'][-1]
        # The contract address is often in topics[1] or topics[2]
        if 'topics' in last_log and len(last_log['topics']) > 1:
            for i in range(1, len(last_log['topics'])):
                try:
                    # Convert bytes to address format
                    potential_address = '0x' + last_log['topics'][i].hex()[-40:]
                    if w3.is_address(potential_address):
                        print(f"Found potential token address in log topics: {potential_address}")
                        return potential_address
                except Exception as e:
                    continue
    
    # Approach 3: Check if the address is in the data field
    for log in tx_receipt['logs']:
        if 'data' in log and len(log['data']) >= 42:  # At least enough bytes for an address
            try:
                # Try to extract address from data
                data_hex = log['data'].hex()
                # Look for patterns that might be addresses (20 bytes)
                for i in range(0, len(data_hex) - 40, 2):
                    potential_address = '0x' + data_hex[i:i+40]
                    if w3.is_address(potential_address):
                        print(f"Found potential token address in log data: {potential_address}")
                        return potential_address
            except Exception as e:
                continue
    
    print("Could not find token contract address in transaction logs")
    return None

def mint_auction_item(token_address, metadata, sender_account, to_address=None):
    """Mint a new token with metadata and return its ID"""
    # Convert token_address to checksum address
    token_address = w3.to_checksum_address(token_address)
    
    # Create contract instance
    token_contract = w3.eth.contract(
        address=token_address,
        abi=auction_item_erc721_abi
    )
    
    # Use provided address or default to sender account
    mint_to_address = to_address if to_address else sender_account.address
    # Convert mint_to_address to checksum address if it's not None
    if mint_to_address:
        mint_to_address = w3.to_checksum_address(mint_to_address)
    
    # Mint token to the specified address
    nonce = w3.eth.get_transaction_count(sender_account.address)
    
    txn = token_contract.functions.mintWithMetadata(
        mint_to_address,
        metadata["name"],
        metadata["description"],
        metadata["image"],
        metadata["termsOfService"],
        metadata.get("supplementalImages", [])
    ).build_transaction({
        'from': sender_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = sender_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Get the token ID from the event
    token_id = None
    for log in tx_receipt['logs']:
        try:
            event = token_contract.events.Transfer().process_log(log)
            if event and event['args']['from'] == '0x0000000000000000000000000000000000000000':
                token_id = event['args']['tokenId']
                print(f"Minted token ID: {token_id} to address: {mint_to_address}")
                break
        except Exception as e:
            continue
    
    if token_id is None:
        print("Failed to get token ID from mint transaction")
        return None
    
    return token_id

def transfer_erc721(token_address, token_id, to_address, sender_account):
    """Transfer an ERC721 token to another address"""
    # Convert addresses to checksum format
    token_address = w3.to_checksum_address(token_address)
    to_address = w3.to_checksum_address(to_address)
    
    # Create contract instance
    token_contract = w3.eth.contract(
        address=token_address,
        abi=auction_item_erc721_abi
    )
    
    nonce = w3.eth.get_transaction_count(sender_account.address)
    
    txn = token_contract.functions.transferFrom(
        sender_account.address,
        to_address,
        token_id
    ).build_transaction({
        'from': sender_account.address,
        'nonce': nonce,
        'gas': 300000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = sender_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Transferred token {token_id} from {sender_account.address} to {to_address}")
    return tx_receipt

def prepare_nft_for_auction(auction_house_address, name, symbol, contract_uri, metadata, sender_account):
    """Create an NFT, set metadata, and transfer to auction house in one function"""
    # Step 1: Create a new NFT contract
    token_address = create_auction_item_erc721(name, symbol, contract_uri, sender_account)
    if not token_address:
        print("Failed to create NFT contract")
        return None, None
    
    # Step 2: Mint a token with metadata
    token_id = mint_auction_item(token_address, metadata, sender_account)
    if not token_id:
        print("Failed to mint token")
        return token_address, None
    
    # Step 3: Transfer the token to the auction house
    # Convert auction_house_address to checksum
    auction_house_address = w3.to_checksum_address(auction_house_address)
    
    transfer_receipt = transfer_erc721(token_address, token_id, auction_house_address, sender_account)
    if not transfer_receipt:
        print("Failed to transfer token to auction house")
        return token_address, token_id
    
    print(f"NFT prepared for auction: Contract {token_address}, Token ID {token_id}")
    return token_address, token_id

def create_auction(auction_house_contract, token_address, token_id, start_time, reserve_price, 
                  duration, affiliate_fee, arbiter_address, escrow_factory_address, 
                  sender_account, payment_token=None, premium=False, premium_rate=0, 
                  min_bid_increment=500, time_extension=900):
    """Create a new auction"""
    # Convert addresses to checksum format
    token_address = w3.to_checksum_address(token_address)
    arbiter_address = w3.to_checksum_address(arbiter_address)
    escrow_factory_address = w3.to_checksum_address(escrow_factory_address)
    
    if payment_token is None:
        payment_token = "0x0000000000000000000000000000000000000000"  # ETH
    else:
        payment_token = w3.to_checksum_address(payment_token)
        
    # Print auction parameters for debugging
    print("\n=== Creating Auction with Parameters ===")
    print(f"Token Address: {token_address}")
    print(f"Token ID: {token_id}")
    print(f"Start Time: {start_time} ({time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))})")
    print(f"Reserve Price: {w3.from_wei(reserve_price, 'ether')} ETH")
    print(f"Duration: {duration} seconds ({duration/86400:.2f} days)")
    print(f"Affiliate Fee: {affiliate_fee} basis points ({affiliate_fee/100:.2f}%)")
    print(f"Arbiter: {arbiter_address}")
    print(f"Escrow Factory: {escrow_factory_address}")
    print(f"Payment Token: {payment_token}")
    print(f"Premium: {premium}")
    print(f"Premium Rate: {premium_rate}")
    print(f"Min Bid Increment: {min_bid_increment} basis points ({min_bid_increment/100:.2f}%)")
    print(f"Time Extension: {time_extension} seconds")
    
    try:
        nonce = w3.eth.get_transaction_count(sender_account.address)
        print(f"Sender Account: {sender_account.address}")
        print(f"Nonce: {nonce}")
        
        txn = auction_house_contract.functions.createAuction(
            token_address,
            token_id,
            start_time,
            reserve_price,
            duration,
            affiliate_fee,
            arbiter_address,
            escrow_factory_address,
            payment_token,
            premium,
            premium_rate,
            min_bid_increment,
            time_extension
        ).build_transaction({
            'from': sender_account.address,
            'nonce': nonce,
            'gas': 5000000,
            'gasPrice': w3.eth.gas_price,
        })
        
        print(f"Transaction built successfully")
        
        signed_txn = sender_account.sign_transaction(txn)
        print(f"Transaction signed successfully")
        
        tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
        print(f"Transaction sent with hash: {tx_hash.hex()}")
        
        print(f"Waiting for transaction receipt...")
        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Transaction mined in block {tx_receipt['blockNumber']}")
        print(f"Gas used: {tx_receipt['gasUsed']}")
        print(f"Status: {'Success' if tx_receipt['status'] == 1 else 'Failed'}")
        
        # Print all logs for debugging
        print("\nTransaction logs:")
        for i, log in enumerate(tx_receipt['logs']):
            print(f"Log {i}:")
            print(f"  Address: {log['address']}")
            print(f"  Topics: {[topic.hex() for topic in log['topics']]}")
            print(f"  Data: {log['data']}")
        
        # Process logs to get the auction ID
        auction_id = None
        for log in tx_receipt['logs']:
            try:
                event = auction_house_contract.events.AuctionCreated().process_log(log)
                if event:
                    auction_id = event['args']['auctionId']
                    print(f"\nAuction created with ID: {auction_id}")
                    
                    # Print all event arguments
                    print("Event arguments:")
                    for arg_name, arg_value in event['args'].items():
                        print(f"  {arg_name}: {arg_value}")
                    
                    return auction_id
            except Exception as e:
                print(f"Error processing log: {e}")
                continue
        
        if auction_id is None:
            print("\nNo AuctionCreated event found in the transaction logs")
            
            # Try to extract auction ID from other events or logs
            print("Attempting to extract auction ID from other logs...")
            for log in tx_receipt['logs']:
                # Look for topics that might contain the auction ID
                if len(log['topics']) > 1:
                    try:
                        # The second topic often contains the ID in events
                        potential_id = int(log['topics'][1].hex(), 16)
                        print(f"Found potential auction ID: {potential_id}")
                        return potential_id
                    except Exception as e:
                        continue
            
            return None
            
    except Exception as e:
        print(f"\n=== ERROR CREATING AUCTION ===")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {str(e)}")
        
        # If it's a contract revert, try to decode the reason
        if hasattr(e, 'args') and len(e.args) > 0:
            print(f"Error args: {e.args}")
        
        return None

def place_bid(auction_house_contract, auction_id, bidder_account, affiliate_address, message, bid_amount):
    """Place a bid on an auction"""
    # Convert affiliate_address to checksum
    affiliate_address = w3.to_checksum_address(affiliate_address)
    
    nonce = w3.eth.get_transaction_count(bidder_account.address)
    
    txn = auction_house_contract.functions.createBid(
        auction_id,
        affiliate_address,
        message,
        bid_amount
    ).build_transaction({
        'from': bidder_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': w3.eth.gas_price,
        'value': bid_amount  # For ETH bids
    })
    
    signed_txn = bidder_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Placed bid of {w3.from_wei(bid_amount, 'ether')} ETH on auction {auction_id}")
    return tx_receipt

def end_auction(auction_house_contract, auction_id, sender_account):
    """End an auction"""
    nonce = w3.eth.get_transaction_count(sender_account.address)
    
    txn = auction_house_contract.functions.endAuction(
        auction_id
    ).build_transaction({
        'from': sender_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = sender_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Ended auction {auction_id}")
    return tx_receipt

In [9]:
# Step 1: Create a new auction house
auction_house_address = create_auction_house(
    "Test Auction House",
    "https://example.com/image.png",
    "A test auction house for NFTs",
    30 * 24 * 60 * 60  # 30 days settlement deadline
)

print(f"New auction house created at: {auction_house_address}")

# Create auction house contract instance
auction_house = w3.eth.contract(
    address=auction_house_address,
    abi=auction_house_abi
)

Transaction receipt: AttributeDict({'blockHash': HexBytes('0xa10207e6647fcfb9bea7c88ceafa14a99e41d0df81d050bebc874168cec398a8'), 'blockNumber': 29504271, 'contractAddress': None, 'cumulativeGasUsed': 32377739, 'effectiveGasPrice': 1850634, 'from': '0x9f4640d04371ff6b7886ade5323746388107723a', 'gasUsed': 2707175, 'l1BaseFeeScalar': '0x8dd', 'l1BlobBaseFee': '0xfca5f7d', 'l1BlobBaseFeeScalar': '0x101c12', 'l1Fee': '0xad7ecc296', 'l1GasPrice': '0x1ac674b6', 'l1GasUsed': '0x9d5', 'logs': [AttributeDict({'address': '0x991cEf5027997d248dAA544FC2434E45Dd98A199', 'topics': [HexBytes('0xd572739fdc5a0a4fcfa09f49f2af7f1df24c6506fa3de300a41e524c93fcdd93'), HexBytes('0x00000000000000000000000079f3f47543c56e2410d7eab88615f6e31d452d3d'), HexBytes('0x0000000000000000000000009f4640d04371ff6b7886ade5323746388107723a')], 'data': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000e000000000000000000000000000000000000000

In [10]:
# Step 2: Create an NFT contract for multiple auctions

x= create_auction_item_erc721(
    "Test NFT",
    "TEST",
    "https://example.com/nft.json"
)


Transaction receipt logs:
Log 0: AttributeDict({'address': '0xe0CAAeE154CdD3a2d0e2700c5e476d15ffbA03a2', 'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), HexBytes('0x0000000000000000000000006e1e335f48cde44119361e2cfddda40ed3bd41cb')], 'data': HexBytes('0x'), 'blockNumber': 29504273, 'transactionHash': HexBytes('0x1c5b857e61245803802ca87a17f080fc414cdf8d08446e83bc0d43980a96850f'), 'transactionIndex': 160, 'blockHash': HexBytes('0xa14e293ea7f6a31a703c93c6010dc8c216f9303a4d5f9c3b677f411d9d3ff2a9'), 'logIndex': 532, 'removed': False})
Log 1: AttributeDict({'address': '0xe0CAAeE154CdD3a2d0e2700c5e476d15ffbA03a2', 'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'), HexBytes('0x0000000000000000000000006e1e335f48cde44119361e2cfddda40ed3bd41cb'), HexBytes('0x0000000000000000000000009f4640d04371ff6b7886ade5323746388107723a')], 'data': He

In [18]:
x = '0xe0CAAeE154CdD3a2d0e2700c5e476d15ffbA03a2'

In [19]:
# step 3 mint the item to the auction house
mint_auction_item(x, {"name": "Test Item", "description": "A test item", "image": "https://example.com/nft.png", "termsOfService": "https://example.com/terms.txt"})



Minted token ID: 1 to address: 0x9f4640d04371ff6b7886ade5323746388107723a


1

In [26]:
#create an auction
# Create an auction with affiliate fee set to 100
auction_id = create_auction(
    auction_house,  # Use the contract instance, not the address
    x,  # NFT contract address
    1,  # Token ID
    int(time.time()+60),  # 60 seconds from now
    w3.to_wei(0.0001, 'ether'),  # Reserve price
    7 * 24 * 60 * 60,  # Duration (7 days in seconds)
    100,  # Affiliate fee
    arbiter_account.address,  # Arbiter address
    os.getenv('AFFILIATE_ESCROW_FACTORY_ADDRESS')  # Escrow factory address
)

print(f"Auction created with ID: {auction_id}")



=== Creating Auction with Parameters ===
Token Address: 0xe0CAAeE154CdD3a2d0e2700c5e476d15ffbA03a2
Token ID: 1
Start Time: 1745856953 (2025-04-28 16:15:53)
Reserve Price: 0.0001 ETH
Duration: 604800 seconds (7.00 days)
Affiliate Fee: 100 basis points (1.00%)
Arbiter: 0xE360EE830BB3dF8Ab17C277F1802a2e861850b9B
Escrow Factory: 0xE07c41Bc76A8B56ad7E996cF60A3dDeD96ca575D
Payment Token: 0x0000000000000000000000000000000000000000
Premium: False
Premium Rate: 0
Min Bid Increment: 500 basis points (5.00%)
Time Extension: 900 seconds
Seller Account: 0xE360EE830BB3dF8Ab17C277F1802a2e861850b9B
Nonce: 635
Transaction built successfully
Transaction signed successfully
Transaction sent with hash: 0xce6c74128ad73b240ba59f734e7c3d5fdbce73b92aa4758a2cfbb761f40e79d9
Waiting for transaction receipt...
Transaction mined in block 29533776
Gas used: 32219
Status: Failed

Transaction logs:

No AuctionCreated event found in the transaction logs
Attempting to extract auction ID from other logs...
Auction crea

In [7]:
#load a list of transactions from lw_3171415_3399531.txt
with open('lw_3171415_3399531.txt', 'r') as file:
    transactions = file.readlines()

#for each transaction, place a bid
for transaction in transactions:
    place_bid(auction_house, auction_id, bidder_account, affiliate_address, message, bid_amount)
